In [ ]:
import requests 
from bs4 import BeautifulSoup

In [ ]:
import time

In [ ]:
from tqdm import tqdm

In [ ]:
import pandas as pd

In [ ]:
url_start ="https://onehu.xyz"
start_idx =2
end_idx =234

In [ ]:
def get_info(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content,'html.parser')
    novel_lists = soup.find_all(class_ = "row")
    
    res =[]
    for item in novel_lists:
        # Find elements
        header_text = item.find(class_="index-header").get_text(strip=True)
        header_href = item.find(class_="index-header").find('a')['href']
        excerpt_text = item.find(class_="index-excerpt").get_text(strip=True)
        date_text = item.find("time").get_text(strip=True)
        category_text = item.find(class_="category-chain-item").get_text(strip=True)
        tag_text = item.find_all("div", class_="post-meta")[-1].get_text(strip=True)
        res.append([header_text,header_href,excerpt_text,date_text,category_text,tag_text])
    cols =['title','url','abstract','date','category','tag']
    df =pd.DataFrame(res,columns=cols)
    df.drop_duplicates(inplace=True)
#     df.to_csv(save_path,index=False)
    return df

In [ ]:
articles_df = get_info(url_start)
for i in tqdm(range(2,307)):
    url =f"https://onehu.xyz/page/{i}/#board"
    page_df = get_info(url)
    articles_df =pd.concat([articles_df,page_df])
#     print(i)

In [ ]:
articles_df.to_csv("./data/articles_df.csv")

### get the article


In [ ]:
articles_df =pd.read_csv('./data/articles_df.csv')

In [ ]:
from glob import glob
import os

In [ ]:
used_items =[]
error_items =[]

In [ ]:
articles_df['txt_path'] =None

In [ ]:
n=0
for idx,row in tqdm(articles_df.iterrows()):
    n +=1
    full_url = f"https://onehu.xyz{row.url}"
    resp = requests.get(full_url)
    page_soup = BeautifulSoup(resp.content,'html.parser')
    content = page_soup.find(class_ ="markdown-body")
    if content is None:
        continue
    folder =f"./data/{row.url[1:11]}"
    if os.path.exists(folder):
        pass
    else:
        os.system(f" mkdir -p {folder}")
        print("folder does not exist, create a new folder ",folder)
    txt_path =f'{folder}/{n-1}.txt'
    articles_df.loc[n-1,'txt_path']=txt_path
#     print("save the content")
    with open (txt_path,'w') as f:
        f.write(content.get_text())
#         break
#     break

### data clean

In [ ]:
articles_df =articles_df[["title",'url','abstract','date','category','tag','txt_path']]
articles_df.drop_duplicates(inplace=True)

In [ ]:
df_clean = articles_df[articles_df.txt_path.isna()==False]

In [ ]:
df_clean.drop_duplicates(subset='abstract',inplace=True)

In [ ]:
df_clean['txt_path'] = df_clean['txt_path'].apply(lambda x : x.replace('./data/','').replace('.txt',''))

In [ ]:
df_clean =df_clean[['title', 'url', 'abstract', 'date', 'category',
       'tag', 'txt_path']]

In [ ]:
df_clean['tag'] =df_clean['tag'].apply(lambda x : x.replace("#",""))

In [ ]:
df_clean.to_csv('./data/articles_df_clean.csv')

### category

In [ ]:
category_list =[]

category_set = df_clean['category'].unique().tolist()


for c in category_set:
    sub_df = df_clean[df_clean['category']==c]
    cnt = len(sub_df)
    lists = sub_df[['title','txt_path']].to_dict(orient='records')[:11]
    category_list.append({"category":c,"count":cnt,"lists":lists})

In [ ]:
sorted_data = sorted(category_list, key=lambda x: x['count'], reverse=True)

In [ ]:
import json

In [ ]:
with open('./data/category_lists.json', 'w', encoding='utf-8') as f:
    json.dump(sorted_data, f, ensure_ascii=False, indent=4)

### search

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df_clean =pd.read_csv('./data/articles_df_clean.csv')

In [ ]:
df_clean['search_keywords'] = df_clean.apply(lambda x : ''.join([i for i in x[['title','category','abstract']]]),axis=1)

In [ ]:
df_clean['search_NER'] = df_clean['search_keywords'].apply(lambda x :set([i for i in x]))

In [ ]:
df_clean[['title', 'url', 'abstract', 'date', 'category', 'tag',
       'txt_path', 'search_NER']].to_csv('./data/articles_df_clean.csv',index=False)

In [3]:
df =df_clean

In [4]:
def search_match(s1_set,s2_set):
    return len(s1_set&s2_set)

In [8]:
df['tmp'] = df.apply(lambda x : search_match(set("假千金"),set(x['search_NER'])),axis=1)
sub_df = df[df['tmp']>=1]
sub_df =sub_df.sort_values(by='tmp', ascending=False)
sub_df =sub_df[:10]
# print(sub_df)

In [10]:
sub_df =sub_df.sort_values(by='tmp', ascending=False)

In [12]:
sub_df =sub_df[:10]

In [13]:
sub_df

,title,url,abstract,date,category,tag,txt_path,search_NER,tmp
0,10. 假千金她不干了,/2024/11/11/10%E5%81%87%E5%8D%83%E9%87%91%E5%A...,假千金她不干了 春风过境，爱意滋生 我是个假千金。 对，没错，抱错二十年，真千金她回来了！ ...,2024-11-11,春风过境，爱意滋生,回答,2024/11/11/0,"{'贬', '袋', '十', '错', '三', '两', '是', '。', '二', ...",3
1989,24. 真千金能看见鬼,/2023/11/04/24%E7%9C%9F%E5%8D%83%E9%87%91%E8%8...,真千金能看见鬼 低调厚爱：请星星捎信告白 我是真千金，刚被认回豪门。 绿茶假千金抱着我的亲哥...,2023-11-04,低调厚爱：请星星捎信告白,刺客爱人：甜虐反转的古言故事,2023/11/04/2116,"{'被', '在', '十', '八', '是', '。', '金', '继', '房', ...",3
5285,真假千金的较量,/2022/09/04/%E7%9C%9F%E5%81%87%E5%8D%83%E9%87%...,大学舍友冒充 S 市首富女儿。 在一群马屁精的怂恿下。 到处嚣张跋扈。 地下大佬的儿子惹到了...,2022-09-04,zhihu,杂志,2022/09/04/5619,"{'被', '友', '在', '知', 'z', '道', '录', '跋', '是', ...",3
3180,亲情降温,/2023/05/17/%E4%BA%B2%E6%83%85%E9%99%8D%E6%B8%A9,首富夫妇再次为了假千金，不来医院探望我的时候。 我彻底死心了。 「系统，让癌细胞扩散吧，我想...,2023-05-17,zhihu,蝉声唱（已完结）,2023/05/17/3350,"{'在', '知', 'z', '道', '壁', '是', '。', '⼀', '金', ...",3
4796,真千金她好飒,/2022/11/24/%E7%9C%9F%E5%8D%83%E9%87%91%E5%A5%...,我是被抱错的真千金。 亲生父母找到我接我回家。 假千金连夜带着男朋友回国，指着我说： 「妈，...,2022-11-24,zhihu,杂志,2022/11/24/5052,"{'被', '友', '在', 'z', '凑', '错', '是', '苦', '。', ...",3
4641,有没有真假千金小姐的爽文,/2023/01/01/%E6%9C%89%E6%B2%A1%E6%9C%89%E7%9C%...,一星期前，真正的程家大小姐找上门来。 我才知道，自己是个冒牌货。 但我的亲生父母已经死了，程...,2023-01-01,zhihu,回答,2023/01/01/4884,"{'知', 'z', '道', '是', '。', '金', '注', '混', '候', ...",3
1533,28. 假千金霸凌我,/2024/02/01/28%E5%81%87%E5%8D%83%E9%87%91%E9%9...,假千金霸凌我 爆裂出击：燃烧吧！少女力 曾经霸凌我的人居然是我爸抱错了的女儿。 其实我才应该...,2024-02-01,爆裂出击：燃烧吧！少女力,帝心难测(已完结),2024/02/01/1624,"{'被', '在', '久', '错', '是', '。', '若', '居', '金', ...",3
4778,有哪些女主比较清醒的小说？,/2022/11/28/%E6%9C%89%E5%93%AA%E4%BA%9B%E5%A5%...,我是个假千金，我一早就知道。 我甚至还劝他们去做个亲子鉴定，但没人信啊。 直到真千金回来，我...,2022-11-28,zhihu,杂志,2022/11/28/5033,"{'在', '并', '知', 'z', '道', '留', '甚', '错', '勾', ...",3
4657,有没有真假千金小姐的爽文,/2022/12/31/%E6%9C%89%E6%B2%A1%E6%9C%89%E7%9C%...,姐姐要去认亲。 她说，等她当上郡主，我每天都能有糖糕吃。 姐姐真是世上对我最好的人了。 可是...,2022-12-31,zhihu,回答,2022/12/31/4902,"{'被', '袋', 'z', '十', '八', '府', '是', '。', '金', ...",3
3089,42. 末世后假千金回村种田了,/2023/06/03/42%E6%9C%AB%E4%B8%96%E5%90%8E%E5%8...,末世后假千金回村种田了 非分之想：恰情动时见你 我在沈家待了十八年，他们突然说我是假千金。 ...,2023-06-03,非分之想：恰情动时见你,杂志,2023/06/03/3255,"{'在', '知', '十', '田', '道', '八', '恰', '是', '。', ...",3
